In [1]:
import numpy as np
import matplotlib.pyplot as plt
import lhapdf
import pandas as pd


NNPDF4_nlo = lhapdf.mkPDF('NNPDF40_nlo_as_01180')
data = pd.read_csv("../E288.csv")
alpha = 1/137

mm = 0.5

def pdf(pdfset, flavor, x, QQ):
    return pdfset.xfxQ(flavor, x, QQ)

def S(k):
    return ((k**2)/(mm*np.pi))*np.exp(-(k**2)/mm)

def fDNNQ(QM, b=0.5):
    return np.exp(-b * QM)

def compute_A(x1, x2, qT, QM):
    f_u_x1 = pdf(NNPDF4_nlo, 2, x1, QM) 
    f_ubar_x2 = pdf(NNPDF4_nlo, -2, x2, QM)
    f_u_x2 = pdf(NNPDF4_nlo, 2, x2, QM)
    f_ubar_x1 = pdf(NNPDF4_nlo, -2, x1, QM)

    # Sk_contribution = (1/2)*(np.pi)*(np.exp(-qT*qT/2))
    Sk_contribution = (8*mm*mm + qT*qT*qT*qT)/(32*np.pi*mm)*(np.exp(-(qT*qT)/(2*mm)))

    fDNN_contribution = fDNNQ(QM)

    ux1ubarx2_term = x1*x2*f_u_x1*f_ubar_x2*Sk_contribution
    ubarx1ux2_term = x2*x1*f_u_x2*f_ubar_x1*Sk_contribution
    FUU = (ux1ubarx2_term + ubarx1ux2_term) * fDNN_contribution
    cross_section =  FUU*qT*((4*np.pi*alpha)**2)/(9*QM*QM*QM)
    return cross_section


x1_values = data['xA'].values
x2_values = data['xB'].values
qT_values = data['PT'].values
QM_values = data['QM'].values


A_values = np.array([
    compute_A(x1, x2, qT, QM)
    for x1, x2, qT, QM in zip(x1_values, x2_values, qT_values, QM_values)
])

results_df = pd.DataFrame({
    'x1': x1_values,
    'x2': x2_values,
    'qT': qT_values,
    'QM': QM_values,
    'A': A_values
})

results_df.to_csv("pseudodataE288_BQM_B2.csv", index=False)
print("Computed A values saved to A_for_E288kinematics.csv")

LHAPDF 6.5.4 loading /home/ishara/LHAPDF/LHAPDF-install/share/LHAPDF/NNPDF40_nlo_as_01180/NNPDF40_nlo_as_01180_0000.dat
NNPDF40_nlo_as_01180 PDF set, member #0, version 1; LHAPDF ID = 331700
Computed A values saved to A_for_E288kinematics.csv


/home/ishara/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

def create_folders(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created successfully!")
    else:
        print(f"Folder '{folder_name}' already exists!")

models_folder = 'Models_withLHAPDF'
results_folder = 'Results_withLHAPDF'
loss_plot_folder = 'Loss_Plots'
create_folders(models_folder)
create_folders(results_folder)
create_folders(loss_plot_folder)

alpha = 1/137

data = pd.read_csv("pseudodataE288_BQM_B2.csv")
x1_values = tf.constant(data['x1'].values, dtype=tf.float32)
x2_values = tf.constant(data['x2'].values, dtype=tf.float32)
qT_values = tf.constant(data['qT'].values, dtype=tf.float32)
QM_values = tf.constant(data['QM'].values, dtype=tf.float32)
A_true_values = tf.constant(data['A'].values, dtype=tf.float32)

def DNNQ():
    return models.Sequential([
        layers.Input(shape=(1,)), 
        layers.Dense(100, activation='relu6'),
        layers.Dense(300, activation='relu6'),
        layers.Dense(300, activation='relu6'),
        layers.Dense(250, activation='relu6'),
        layers.Dense(250, activation='relu6'),
        layers.Dense(1, activation='exponential')
    ])

def pdf(pdfset, flavor, x, QQ):
    return pdfset.xfxQ(flavor, x, QQ)

def custom_loss(dnnQ, A_true, x1, x2, qT, QM):
    dnnQinputs = tf.reshape(QM, (-1, 1))
    dnnQvals = dnnQ(dnnQinputs)

    f_u_x1 = tf.constant(pdf(NNPDF4_nlo, 2, x1, QM), dtype=tf.float32)
    f_ubar_x2 = tf.constant(pdf(NNPDF4_nlo, -2, x2, QM), dtype=tf.float32)
    f_u_x2 = tf.constant(pdf(NNPDF4_nlo, 2, x2, QM), dtype=tf.float32)
    f_ubar_x1 = tf.constant(pdf(NNPDF4_nlo, -2, x1, QM), dtype=tf.float32)

    pi = tf.constant(np.pi, dtype=tf.float32)
    Sk_contribution = (8*mm*mm + qT*qT*qT*qT)/(32*pi*mm)*(tf.exp(-(qT*qT)/(2*mm)))

    ux1ubarx2_term = x1 * x2 * f_u_x1 * f_ubar_x2 * Sk_contribution
    ubarx1ux2_term = x2 * x1 * f_u_x2 * f_ubar_x1 * Sk_contribution
    FUU = ux1ubarx2_term + ubarx1ux2_term
    cross_section = FUU * qT * ((4 * np.pi * alpha) ** 2) / (9 * QM * QM * QM) * dnnQvals
    temploss = tf.abs(cross_section - A_true)
    loss = tf.reduce_mean(temploss)  # MAE loss
    return loss

num_models = 50
initial_lr = 0.1  
lr_factor = 0.9  
patience = 100  
best_loss = 5.0e-11 
counter = 0  

for i in range(1, num_models + 1):
    dnnQ = DNNQ()
    optimizer = tf.keras.optimizers.Adam(learning_rate=initial_lr)
    epochs = 10000
    print_epochs = 100
    
    losses = []
    lr_values = [initial_lr]
    lr_values_4plot = [initial_lr]
    lr_epochs = []

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            loss = custom_loss(dnnQ, A_true_values, x1_values, x2_values, qT_values, QM_values)
        grads = tape.gradient(loss, dnnQ.trainable_variables)
        optimizer.apply_gradients(zip(grads, dnnQ.trainable_variables))
        losses.append(loss.numpy())

        # Learning rate adaptation logic
        if loss.numpy() < best_loss:
            best_loss = loss.numpy()
            counter = 0  # Reset patience counter
        else:
            counter += 1  # Increase counter if no improvement
        
        # Reduce LR if no improvement for 'patience' epochs
        if counter >= patience:
            new_lr = optimizer.learning_rate.numpy() * lr_factor
            optimizer.learning_rate.assign(new_lr)
            lr_values.append(new_lr)
            counter = 0  # Reset counter after reducing LR
            print(f"Epoch {epoch + 1}: Reducing learning rate to {new_lr:.6f}")

        if epoch % print_epochs == 0 or epoch == epochs - 1:
            print(f"Model {i} - Epoch {epoch + 1}/{epochs}, Loss: {loss.numpy():.3e}, LR: {optimizer.learning_rate.numpy():.6f}")
    
    model_path = os.path.join(models_folder, f'DNNQ_model_{i}.h5')
    dnnQ.save(model_path)
    print(f"Model {i} saved successfully at {model_path}!")
    
    # Plot Loss over Epochs
    plt.figure(figsize=(10, 6))
    plt.plot(range(epochs), losses, label='Loss', color='b')
    plt.title(f'Model {i} Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    loss_plot_path = os.path.join(loss_plot_folder, f'loss_plot_model_{i}.pdf')
    plt.savefig(loss_plot_path)
    print(f"Loss plot for Model {i} saved successfully at {loss_plot_path}!")
    

#     # Plot Learning Rate Decay
#     plt.figure(figsize=(10, 6))
#     #plt.plot(range(0, epochs, patience), lr_values, label='Learning Rate', color='r')
#     plt.plot(lr_epochs, lr_values_4plot, label='Learning Rate', color='r')
#     plt.title(f'Model {i} Learning Rate Decay')
#     plt.xlabel('Epochs')
#     plt.ylabel('Learning Rate')
#     plt.legend()
#     plt.grid(True)

#     lr_plot_path = os.path.join(loss_plot_folder, f'lr_plot_model_{i}.pdf')
#     plt.savefig(lr_plot_path)
#     print(f"Learning rate plot for Model {i} saved successfully at {lr_plot_path}!")


Folder 'Models_withLHAPDF' already exists!
Folder 'Results_withLHAPDF' already exists!
Folder 'Loss_Plots' already exists!
Model 1 - Epoch 1/10000, Loss: 4.685e-11, LR: 0.100000
Model 1 - Epoch 101/10000, Loss: 4.320e-11, LR: 0.100000
Model 1 - Epoch 201/10000, Loss: 3.848e-11, LR: 0.100000
Model 1 - Epoch 301/10000, Loss: 3.422e-11, LR: 0.100000
